<a href="https://colab.research.google.com/github/arkeodev/time-series/blob/main/Statistical_Time_Series_Analysis/14-ARIMAX-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 14. ARIMAX Models

## 1. Introduction

The ARIMAX model stands for Autoregressive Integrated Moving Average with eXogenous inputs model. It is an extension of the ARIMA model, which includes external factors.

In ARIMAX, the ‘X’ refers to the exogenous variables, which are external factors or covariates that might influence the time series. These variables can include anything relevant that could impact the response variable you are trying to forecast but is not directly part of the series. 

For instance, if you were predicting demand for umbrellas, an exogenous variable might be rainfall. Unlike endogenous variables in ARIMA, exogenous variables are not predicted by the model but are included in the model to help improve the accuracy of predictions.

## 2. Mathematical Notation

In mathematical terms, the ARIMAX model is written as:

$$
\Delta P_t = c + \phi_1 \Delta P_{t-1} + \theta_1 \varepsilon_{t-1} + \beta X_t + \varepsilon_t
$$

Where:
- $ \Delta P_t $ is the differenced time series value at time t.
- $ c $ is a constant.
- $ \phi_1 $ is the coefficient for the first lag of the differenced series.
- $ \theta_1 $ is the coefficient for the first lag of the error term.
- $ \beta $ is the coefficient for the exogenous variable $ X_t $.
- $ \varepsilon_t $ is the error term (residuals of the model).

## 3. Practical Considerations

1. The exogenous variable $ X $ can be:
- A **time-varying measurement** (e.g., temperature, economic indicators).
- A **categorical variable** (e.g., day of the week, event occurrence).
- A **boolean value** (e.g., presence or absence of an event).
- A combination of several different factors.

2. It's important that for ARIMAX,**you must have data** for the exogenous variables for every period you're trying to model.

3. When incorporating exogenous variables, it’s crucial to ensure that these factors are indeed influential to the model and are **not just noise**. Overfitting the model with irrelevant X variables can lead to poor predictive performance.

## 4. Implementation

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------------
# Step 1: Set Seed for Reproducibility
# -----------------------------------------------------------------------------------
# Setting a random seed ensures that the randomly generated values 
# are the same each time you run this script, making your results reproducible.
# -----------------------------------------------------------------------------------
np.random.seed(42)

# -----------------------------------------------------------------------------------
# Step 2: Generate a Series y that Follows an Integrated Process
# -----------------------------------------------------------------------------------
# We aim to mimic an ARIMA(1, 1, 1) process. In such a process:
#   - The series is differenced once (d=1) to become stationary.
#   - There is one AR(1) term, meaning today's value depends on yesterday's value.
#   - There is one MA(1) term, meaning today's value depends on yesterday's shock.
#
# For demonstration, we:
#   - Create AR and MA parameter arrays.
#   - Generate random noise and apply a cumulative sum to create a non-stationary series.
# Note: The direct usage of ar_params and ma_params here is symbolic rather than fully
# generating a strict ARIMA(1,1,1) using those exact parameters with noise. Instead, 
# we simulate an integrated process (random walk-like) to illustrate an ARIMA setup.
# -----------------------------------------------------------------------------------
ar_params = np.array([0.5])    # AR(1) parameter
ma_params = np.array([-0.3])   # MA(1) parameter
ar = np.r_[1, -ar_params]      # Becomes [1, -0.5]
ma = np.r_[1, ma_params]       # Becomes [1, -0.3]

# Here we create a random walk by cumulatively summing white noise.
# The result is a non-stationary series that would require differencing.
y = np.random.normal(loc=0, scale=1, size=100).cumsum()

# -----------------------------------------------------------------------------------
# Step 3: Generate a Synthetic Exogenous Variable x
# -----------------------------------------------------------------------------------
# We create another 100-point series (e.g., representing an external factor like 
# temperature, a stock market index, etc.). 
# This variable is also constructed by taking a cumulative sum of random normal values.
# -----------------------------------------------------------------------------------
x = np.random.normal(loc=0, scale=1, size=100).cumsum()

# -----------------------------------------------------------------------------------
# Step 4: Fit an ARIMAX Model
# -----------------------------------------------------------------------------------
# In statsmodels, the SARIMAX class can be used to specify a model with exogenous variables.
# - order=(1, 1, 1) indicates an ARIMA model with p=1, d=1, q=1.
# - exog=x means we include the exogenous variable in the model, so it becomes ARIMAX.
#
# The exogenous variable, x, is expected to help explain part of the variation in y
# that is not captured by the ARIMA components alone.
# -----------------------------------------------------------------------------------
model = SARIMAX(y, order=(1, 1, 1), exog=x)
results = model.fit()

# -----------------------------------------------------------------------------------
# Step 5: Summarize the Model Results
# -----------------------------------------------------------------------------------
# This includes parameter estimates (for AR, MA, and the exogenous variable's coefficient),
# as well as diagnostics like AIC and BIC for model comparison.
# -----------------------------------------------------------------------------------
print(results.summary())


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.30539D+00    |proj g|=  1.17406D-02

At iterate    5    f=  1.30527D+00    |proj g|=  1.49366D-03

At iterate   10    f=  1.30520D+00    |proj g|=  6.14288D-03

At iterate   15    f=  1.30518D+00    |proj g|=  1.05106D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     15     27      1     0     0   1.051D-05   1.305D+00
  F =   1.3051781967717191     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
        

 This problem is unconstrained.


*ARIMAX vs. SARIMAX:*

While the model is commonly referred to as ARIMAX (ARIMA with exogenous variables), in statsmodels, the SARIMAX class is used because it can handle both seasonal and non-seasonal ARIMA components and exogenous regressors. For a simple ARIMAX (non-seasonal, with exogenous variables), you just set seasonal parameters to zero or omit them.

## 5. Results Interpretation

Here’s a **summary** of the model’s output:

1. **Model & Fit**  
   - **SARIMAX(1,1,1) with exogenous (x1)**  
   - **Converged** in 15 iterations using L-BFGS-B.  
   - **Log Likelihood:** $(-130.518$, **AIC:** 269.036, **BIC:** 279.416.

2. **Key Parameters**  
   - **x1 Coefficient:** $(-0.1323$, $p=0.128$) → Not statistically significant.  
   - **AR(1) Coefficient:** $(0.2546$, $p=0.937$) → Not significant.  
   - **MA(1) Coefficient:** $(-0.2875$, $p=0.928$) → Not significant.  
   - **$(\sigma^2)$ (Error Variance):** $(0.8178$, $p<0.001$) → Significant.

3. **Diagnostics**  
   - **Ljung-Box (Q):** $(p=0.90$) → No significant autocorrelation in residuals.  
   - **Jarque-Bera (JB):** $(p=0.90$) → Residuals appear normally distributed.  
   - **Heteroskedasticity (H):** $(p=0.64$) → No evidence of changing variance.


**Takeaways**

1. **Insignificant AR and MA Terms**  
   - The large standard errors and high p-values suggest that neither AR(1) nor MA(1) terms are strongly identified in this data sample. It could be that the integrated random walk structure is overshadowing the AR/MA components, or there isn’t enough strong signal in 100 points to precisely pin down those parameters.

2. **Non-Significant Exogenous Variable**  
   - The exogenous predictor `x1` has a negative coefficient but also lacks statistical significance. In real-world scenarios, you might want more data or a more relevant exogenous series to see a stronger effect.

3. **Residual Diagnostics**  
   - High p-values in the Ljung-Box and Jarque-Bera tests suggest the residuals are not autocorrelated and look approximately normal, which is generally favorable for a time series model. 

4. **Potential Next Steps**  
   - Try different orders (p, d, q) or add seasonal components if you suspect seasonal patterns.  
   - Increase the dataset size or use a more strongly related exogenous variable if possible.  
   - Compare the AIC/BIC to a simpler model (e.g., random walk with drift, ARIMA(1,1,0), etc.) to see if the exogenous variable meaningfully improves the fit.

Overall, while the model residuals look okay based on diagnostic tests, the AR(1), MA(1), and exogenous coefficients are not significant. This might mean the data (or the way it was generated) does not truly contain strong AR, MA, or exogenous relationships—or that the sample size is too small to detect them precisely.